In [1]:
import os
import sys
code_dir = '/afs/inf.ed.ac.uk/user/s17/s1771906/masters-project/ben-rhodes-masters-project/proposal/code'
code_dir_2 = '/home/ben/ben-rhodes-masters-project/proposal/code'
code_dir_3 = '/afs/inf.ed.ac.uk/user/s17/s1771906/masters-project/ben-rhodes-masters-project/proposal/code/neural_network'
code_dir_4 = '/home/ben/ben-rhodes-masters-project/proposal/code/neural_network'
code_dirs = [code_dir, code_dir_2, code_dir_3, code_dir_4]
for code_dir in code_dirs:
    if code_dir not in sys.path:
        sys.path.append(code_dir)

In [2]:
import numpy as np
import pickle

# my code
from distribution import MissingDataProductOfTruncNormsPosterior, MissingDataProductOfTruncNormNoise
from fully_observed_models import MixtureOfTwoGaussians
from latent_variable_model import MissingDataUnnormalisedTruncNorm
from nce_optimiser import NCEOptimiser
from plot import *
from utils import *
from vnce_optimiser import VemOptimiser, SgdEmStep, MonteCarloVnceLoss

from copy import deepcopy
from matplotlib import pyplot as plt
from matplotlib import rc
from numpy import random as rnd
from scipy.optimize import newton_krylov
from scipy.stats import norm, multivariate_normal
from sklearn.neighbors import KernelDensity as kd

%matplotlib inline
rc('lines', linewidth=1)
rc('font', size=10)
rc('legend', fontsize=10)

In [3]:
# load_path = '/home/ben/ben-rhodes-masters-project/experimental_results/trunc_norm/test/test/'
load_path = '/disk/scratch/ben-rhodes-masters-project/experimental-results/trunc_norm/test/d=2-frac=0.1/'

In [4]:
data_dist = pickle.load(open(os.path.join(load_path, 'data_dist.p'), 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: '/disk/scratch/ben-rhodes-masters-project/experimental-results/trunc_norm/test/d=2-frac=0.1/data_dist.p'

In [ ]:
vnce_model = pickle.load(open(os.path.join(load_path, 'vnce_model.p'), 'rb'))

In [ ]:
nce_model = pickle.load(open(os.path.join(load_path, 'nce_model.p'), 'rb'))

In [ ]:
var_dist = pickle.load(open(os.path.join(load_path, 'var_dist.p'), 'rb'))

In [ ]:
noise = pickle.load(open(os.path.join(load_path, 'noise.p'), 'rb'))

In [ ]:
globals().update(np.load(os.path.join(load_path, 'data.npz')))
globals().update(np.load(os.path.join(load_path, 'theta0_and_theta_true.npz')))

In [ ]:
# contour plot of q(z|x)p_y(x) next to cplot of model 
# do this for two cases: z is first dimension, and z is second dimension
def get_visible_and_latents(U, latent_dim=1):
    visible_dim = int(~latent_dim)
    x = np.zeros_like(U)
    z = np.zeros((1, ) + U.shape)
    
    x[:, visible_dim] = U[:, visible_dim]
    z[0, :, latent_dim] = U[:, latent_dim]
    return x, z

def joint_noise(U):
    x, z = get_visible_and_latents(U)
    return var_dist(z, x) * noise(x)

def joint_noise_rev(U):
    x, z = get_visible_and_latents(U, latent_dim=0)
    return var_dist(z, x) * noise(x)

def eval_vnce_model(U):
    x, z = get_visible_and_latents(U)
    return vnce_model(x, z)

def eval_data_dist(U):
    x, z = get_visible_and_latents(U)
    return data_dist(x, z)
    
def eval_nce_model(U):
    return nce_model(U)

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(25, 5))
axs = axs.ravel()
x_low, x_high, y_low, y_high= -2, 10, -2, 10
plot_2d_density(axs[0], joint_noise, plt.cm.YlOrRd_r, low_x_lim=x_low, up_x_lim=x_high, low_y_lim=y_low, up_y_lim=y_high)
plot_2d_density(axs[1], joint_noise_rev, plt.cm.YlOrRd_r, low_x_lim=x_low, up_x_lim=x_high, low_y_lim=y_low, up_y_lim=y_high)
plot_2d_density(axs[2], eval_vnce_model, plt.cm.YlOrRd_r, low_x_lim=x_low, up_x_lim=x_high, low_y_lim=y_low, up_y_lim=y_high)
plot_2d_density(axs[3], nce_model, plt.cm.YlGn_r, low_x_lim=x_low, up_x_lim=x_high, low_y_lim=y_low, up_y_lim=y_high)
plot_2d_density(axs[4], eval_data_dist, plt.cm.GnBu_r, low_x_lim=x_low, up_x_lim=x_high, low_y_lim=y_low, up_y_lim=y_high)

In [ ]:
noise.alpha

In [ ]:
var_dist.nn.params

In [ ]:
vnce_model.theta

In [ ]:
nce_model.theta

In [ ]:
# old -  0.76872111
# noise.alpha[-2] = 0.7687211

In [11]:
a = np.diag(np.ones(5))
b = np.diag(np.ones(4), 1) * 0.4
c = np.diag(np.ones(4), -1) * 0.4

In [12]:
d = a + b + c
d[0, -1] = 0.4
d[-1, 0] = 0.4
d

array([[ 1. ,  0.4,  0. ,  0. ,  0.4],
       [ 0.4,  1. ,  0.4,  0. ,  0. ],
       [ 0. ,  0.4,  1. ,  0.4,  0. ],
       [ 0. ,  0. ,  0.4,  1. ,  0.4],
       [ 0.4,  0. ,  0. ,  0.4,  1. ]])

In [14]:
L = np.linalg.cholesky(d)

In [16]:
np.dot(L, L.T)

array([[ 1. ,  0.4,  0. ,  0. ,  0.4],
       [ 0.4,  1. ,  0.4,  0. ,  0. ],
       [ 0. ,  0.4,  1. ,  0.4,  0. ],
       [ 0. ,  0. ,  0.4,  1. ,  0.4],
       [ 0.4,  0. ,  0. ,  0.4,  1. ]])

In [17]:
np.linalg.inv(d)

array([[ 1.56565657, -0.70707071,  0.2020202 ,  0.2020202 , -0.70707071],
       [-0.70707071,  1.56565657, -0.70707071,  0.2020202 ,  0.2020202 ],
       [ 0.2020202 , -0.70707071,  1.56565657, -0.70707071,  0.2020202 ],
       [ 0.2020202 ,  0.2020202 , -0.70707071,  1.56565657, -0.70707071],
       [-0.70707071,  0.2020202 ,  0.2020202 , -0.70707071,  1.56565657]])